# 10. 보고서용 정확한 통계 계산

## 목적
- 가설1 검증 보고서에 필요한 모든 통계를 정확히 계산
- 시간대별 평균 지표 계산
- 클러스터별 평균 지표 계산  
- 상권별 평균 지표 계산

## ⚠️ 데이터 구조 주의사항
- **'당월_매출_금액'은 개별 가맹점 매출이 아님**
- **상권_코드 × 기준년월 단위의 상권 매출액**
- 같은 상권_코드에 속한 모든 가맹점이 동일한 값을 공유
- 따라서 "소속 상권의 월별 평균 매출"로 해석해야 함

---

## 1. 데이터 로드 및 전처리

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 원본 데이터 로드
data_path = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1009/빅콘테스트_전체병합데이터_20251008.csv"
df = pd.read_csv(data_path)

print(f"데이터 형태: {df.shape}")
print(f"관측치 수: {len(df):,}개")

데이터 형태: (86263, 188)
관측치 수: 86,263개


## 2. 시간 기반 라벨링

In [2]:
# 날짜 변환
df['기준년월_dt'] = pd.to_datetime(df['기준년월'].astype(str), format='%Y%m', errors='coerce')

# 폐업월 계산
def _to_periodM_from_any(s):
    s = s.astype(str).str.replace(r'\.0$', '', regex=True).str.strip()
    dt = pd.to_datetime(s, errors='coerce')
    return dt.dt.to_period('M')

df['_관측월'] = df['기준년월_dt'].dt.to_period('M')
df['_폐업월'] = _to_periodM_from_any(df['폐업일'])

# 개월차 계산
y1, m1 = df['_폐업월'].dt.year, df['_폐업월'].dt.month
y0, m0 = df['_관측월'].dt.year, df['_관측월'].dt.month
df['_months_to_close'] = (y1 - y0) * 12 + (m1 - m0)

print(f"폐업 가맹점: {df['_폐업월'].notna().sum():,}개")

폐업 가맹점: 2,334개


In [3]:
# 정상영업: 한번도 폐업 기록 없는 가맹점의 최신 데이터
closed_any = df.groupby('가맹점구분번호')['_폐업월'].transform(lambda s: s.notna().any())
normal_latest = (df[~closed_any]
                 .sort_values(['가맹점구분번호', '기준년월_dt'])
                 .groupby('가맹점구분번호', as_index=False)
                 .tail(1)
                 .assign(상태='정상영업'))

print(f"정상영업: {len(normal_latest):,}개")

# 폐업 D-k: 각 가맹점에서 정확히 D-k 개월 전 스냅샷 1건
def _pick_preclose(k):
    sub = df[df['_months_to_close'].eq(k)].copy()
    if sub.empty:
        return sub.assign(상태=f'D-{k}m')
    sub = (sub.sort_values(['가맹점구분번호', '기준년월_dt'])
              .groupby('가맹점구분번호', as_index=False)
              .tail(1))
    sub['상태'] = f'D-{k}m'
    return sub

d12 = _pick_preclose(12)
d9 = _pick_preclose(9)
d6 = _pick_preclose(6)
d3 = _pick_preclose(3)

print(f"D-12m: {len(d12):,}개")
print(f"D-9m: {len(d9):,}개")
print(f"D-6m: {len(d6):,}개")
print(f"D-3m: {len(d3):,}개")

# 통합
ana = pd.concat([normal_latest, d12, d9, d6, d3], ignore_index=True)

print(f"\n통합 데이터: {len(ana):,}개")

정상영업: 4,043개
D-12m: 105개
D-9m: 113개
D-6m: 32개
D-3m: 32개

통합 데이터: 4,325개


## 3. 시간대별 평균 지표 계산

### 📌 해석 주의
- **'소속_상권_매출_억원'**: 해당 가맹점이 속한 상권의 월별 매출액 평균 (개별 가맹점 매출 아님)
- **'1km내_지하철승하차'**: 가맹점 1km 반경 내 지하철 승하차 인원

In [4]:
# 시간대별 주요 지표 평균 계산
time_stats = ana.groupby('상태').agg({
    '당월_매출_금액': 'mean',
    '당월_매출_건수': 'mean',
    '배달매출금액 비율': 'mean',
    '1km내_총지하철승하차': 'mean',
    '가맹점구분번호': 'count'
}).round(2)

time_stats.columns = ['소속_상권_매출_원', '상권_매출건수', '배달비율', '1km내_지하철승하차', '가맹점수']

# 억원 단위 추가
time_stats['소속_상권_매출_억원'] = (time_stats['소속_상권_매출_원'] / 100000000).round(2)

print("="*80)
print("시간대별 평균 지표")
print("="*80)
print(time_stats[['가맹점수', '소속_상권_매출_억원', '상권_매출건수', '배달비율', '1km내_지하철승하차']])

print("\n📌 해석:")
print("- '소속_상권_매출_억원': 가맹점이 속한 상권의 월별 매출액 평균 (개별 가맹점 매출 아님)")
print("- '1km내_지하철승하차': 가맹점 중심 1km 반경 내 월별 누적 승하차 인원")

# 정상 대비 배수 계산
if '정상영업' in time_stats.index:
    normal_delivery = time_stats.loc['정상영업', '배달비율']
    print(f"\n정상영업 배달비율: {normal_delivery:.2f}%")
    print("\n정상영업 대비 배달비율 배수:")
    for status in time_stats.index:
        if status != '정상영업':
            ratio = time_stats.loc[status, '배달비율'] / normal_delivery
            print(f"  {status}: {ratio:.2f}배")

시간대별 평균 지표
       가맹점수  소속_상권_매출_억원   상권_매출건수   배달비율  1km내_지하철승하차
상태                                                    
D-12m   105         5.86  17095.17  15.64   4808731.99
D-3m     32        10.08  40291.80  25.12   5010425.94
D-6m     32         9.21  31400.24  15.88   5067467.28
D-9m    113         7.01  22395.81  14.67   4772424.33
정상영업   4043         8.14  30363.04  10.47   4648461.80

📌 해석:
- '소속_상권_매출_억원': 가맹점이 속한 상권의 월별 매출액 평균 (개별 가맹점 매출 아님)
- '1km내_지하철승하차': 가맹점 중심 1km 반경 내 월별 누적 승하차 인원

정상영업 배달비율: 10.47%

정상영업 대비 배달비율 배수:
  D-12m: 1.49배
  D-3m: 2.40배
  D-6m: 1.52배
  D-9m: 1.40배


## 4. 클러스터 정보 병합

In [5]:
# 클러스터링 결과 로드
cluster_path = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/3_가설1분석/클러스터링_결과_완전판.csv"
cluster_df = pd.read_csv(cluster_path)

# 병합
ana_with_cluster = ana.merge(cluster_df[['가맹점구분번호', 'cluster']], on='가맹점구분번호', how='left')

print(f"클러스터 정보 병합 완료: {len(ana_with_cluster):,}개")
print(f"\n클러스터별 분포:")
print(ana_with_cluster['cluster'].value_counts().sort_index())

클러스터 정보 병합 완료: 4,773개

클러스터별 분포:
cluster
0    4125
1     200
2     448
Name: count, dtype: int64


## 5. 클러스터별 평균 지표 계산

In [6]:
# 클러스터별 주요 지표 평균
cluster_stats = ana_with_cluster.groupby('cluster').agg({
    '가맹점구분번호': 'count',
    '당월_매출_금액': 'mean',
    '1km내_총지하철승하차': 'mean',
    '배달매출금액 비율': 'mean',
    '남성 60대이상 고객 비중': 'mean',
    '여성 20대이하 고객 비중': 'mean',
    '남성 20대이하 고객 비중': 'mean',
    '여성 30대 고객 비중': 'mean',
    '남성 50대 고객 비중': 'mean'
}).round(2)

cluster_stats.columns = ['가맹점수', '소속_상권_매출_원', '1km내_지하철승하차', '배달비율', 
                          '남성60대이상', '여성20대이하', '남성20대이하', '여성30대', '남성50대']

# 억원 단위 추가
cluster_stats['소속_상권_매출_억원'] = (cluster_stats['소속_상권_매출_원'] / 100000000).round(2)

print("="*80)
print("클러스터별 평균 지표")
print("="*80)
print(cluster_stats[['가맹점수', '소속_상권_매출_억원', '1km내_지하철승하차', '배달비율', 
                      '남성60대이상', '여성20대이하', '여성30대']])

print("\n📌 해석:")
print("- '소속_상권_매출_억원': 클러스터 내 가맹점들이 속한 상권의 평균 매출")

# 클러스터 0 대비 배수 계산
if 0 in cluster_stats.index:
    print("\n클러스터 0 대비 비율:")
    for col in ['1km내_지하철승하차', '배달비율']:
        base_val = cluster_stats.loc[0, col]
        print(f"\n{col}:")
        for idx in cluster_stats.index:
            if idx != 0:
                ratio = cluster_stats.loc[idx, col] / base_val
                print(f"  클러스터 {int(idx)}: {ratio:.2f}배")

클러스터별 평균 지표
         가맹점수  소속_상권_매출_억원  1km내_지하철승하차   배달비율  남성60대이상  여성20대이하  여성30대
cluster                                                                
0        4125         8.10   4659073.56  10.63     9.38    10.58  11.82
1         200        11.81   4931418.70  19.33     5.75    12.06  10.96
2         448         5.49   4799417.20  14.87     5.54    14.25  13.31

📌 해석:
- '소속_상권_매출_억원': 클러스터 내 가맹점들이 속한 상권의 평균 매출

클러스터 0 대비 비율:

1km내_지하철승하차:
  클러스터 1: 1.06배
  클러스터 2: 1.03배

배달비율:
  클러스터 1: 1.82배
  클러스터 2: 1.40배


## 6. 상권별 평균 지표 계산

### 📌 주의
- "상권" 컬럼: 지역명 (예: 성수, 뚝섬)
- 하나의 지역명 안에 여러 상권_코드가 존재할 수 있음

In [7]:
# 상권별 전체 평균
region_stats = ana_with_cluster.groupby('상권').agg({
    '가맹점구분번호': 'count',
    '1km내_총지하철승하차': 'mean',
    '배달매출금액 비율': 'mean',
    '남성 60대이상 고객 비중': 'mean',
    '여성 20대이하 고객 비중': 'mean',
    '남성 20대이하 고객 비중': 'mean',
    '여성 30대 고객 비중': 'mean'
}).round(0)

region_stats.columns = ['가맹점수', '1km내_지하철승하차', '배달비율', 
                         '남성60대이상', '여성20대이하', '남성20대이하', '여성30대']

# 20-30대 비중 합계
region_stats['20-30대비중'] = (region_stats['여성20대이하'] + 
                              region_stats['남성20대이하'] + 
                              region_stats['여성30대']).round(0)

print("="*80)
print("상권별 평균 지표")
print("="*80)
print(region_stats[['가맹점수', '1km내_지하철승하차', '배달비율', '남성60대이상', '20-30대비중']])

print("\n주요 상권:")
for region in ['성수', '뚝섬', '왕십리', '한양대', '마장동', '금남시장']:
    if region in region_stats.index:
        print(f"\n{region}:")
        print(f"  - 1km내 지하철승하차: {region_stats.loc[region, '1km내_지하철승하차']:,.0f}명")
        print(f"  - 배달비율: {region_stats.loc[region, '배달비율']:.0f}%")
        print(f"  - 20-30대 비중: {region_stats.loc[region, '20-30대비중']:.0f}%")

상권별 평균 지표
        가맹점수  1km내_지하철승하차  배달비율  남성60대이상  20-30대비중
상권                                                
금남시장     400    2320557.0  21.0     12.0      20.0
답십리      213    1700884.0  10.0     12.0      27.0
뚝섬       657    3090959.0   8.0      5.0      47.0
마장동      449    6549700.0   3.0     18.0      13.0
성수      1128    3674398.0   6.0      7.0      42.0
신금호      138    3223901.0  18.0     12.0      19.0
옥수       152    1702476.0  14.0      9.0      24.0
왕십리     1043    7130234.0  19.0      9.0      28.0
장한평자동차    55    1307942.0   9.0     17.0      20.0
한양대      426    7544560.0  10.0      3.0      59.0
행당       112    4659514.0  19.0      8.0      21.0

주요 상권:

성수:
  - 1km내 지하철승하차: 3,674,398명
  - 배달비율: 6%
  - 20-30대 비중: 42%

뚝섬:
  - 1km내 지하철승하차: 3,090,959명
  - 배달비율: 8%
  - 20-30대 비중: 47%

왕십리:
  - 1km내 지하철승하차: 7,130,234명
  - 배달비율: 19%
  - 20-30대 비중: 28%

한양대:
  - 1km내 지하철승하차: 7,544,560명
  - 배달비율: 10%
  - 20-30대 비중: 59%

마장동:
  - 1km내 지하철승하차: 6,549,700명
  - 배달비율: 3%
  - 20-3

## 7. 상권별 × 클러스터별 교차 분석

In [8]:
# 상권별 클러스터 분포
region_cluster_dist = pd.crosstab(ana_with_cluster['상권'], 
                                   ana_with_cluster['cluster'], 
                                   margins=True,
                                   normalize='index') * 100

region_cluster_dist = region_cluster_dist.round(1)

print("="*80)
print("상권별 클러스터 분포 (%)")
print("="*80)
print(region_cluster_dist)

# 고위험 비율 계산 (클러스터 1, 2)
print("\n고위험 비율 (클러스터 1+2):")
region_cluster_count = pd.crosstab(ana_with_cluster['상권'], ana_with_cluster['cluster'])
region_cluster_count['고위험'] = region_cluster_count.get(1, 0) + region_cluster_count.get(2, 0)
region_cluster_count['전체'] = region_cluster_count.sum(axis=1)
region_cluster_count['고위험비율'] = (region_cluster_count['고위험'] / region_cluster_count['전체'] * 100).round(1)

print(region_cluster_count[['고위험', '전체', '고위험비율']].sort_values('고위험비율', ascending=False))

상권별 클러스터 분포 (%)
cluster     0     1     2
상권                       
금남시장     84.2   6.0   9.8
답십리      84.5   4.7  10.8
뚝섬       89.5   2.1   8.4
마장동      87.5   7.1   5.3
성수       88.7   2.6   8.7
신금호      78.3   0.0  21.7
옥수       97.4   0.0   2.6
왕십리      83.3   5.9  10.7
장한평자동차   80.0  16.4   3.6
한양대      83.3   4.7  12.0
행당       91.1   0.0   8.9
All      86.4   4.2   9.4

고위험 비율 (클러스터 1+2):
cluster  고위험    전체  고위험비율
상권                       
신금호       30   168   17.9
장한평자동차    11    66   16.7
왕십리      174  1217   14.3
한양대       71   497   14.3
금남시장      63   463   13.6
답십리       33   246   13.4
마장동       56   505   11.1
성수       127  1255   10.1
뚝섬        69   726    9.5
행당        10   122    8.2
옥수         4   156    2.6


## 8. 결과 저장

In [9]:
import os

output_dir = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/5_보고서통계"
os.makedirs(output_dir, exist_ok=True)

# 저장
time_stats.to_csv(f"{output_dir}/시간대별_평균지표.csv", encoding='utf-8-sig')
cluster_stats.to_csv(f"{output_dir}/클러스터별_평균지표.csv", encoding='utf-8-sig')
region_stats.to_csv(f"{output_dir}/상권별_평균지표.csv", encoding='utf-8-sig')
region_cluster_count.to_csv(f"{output_dir}/상권별_클러스터분포.csv", encoding='utf-8-sig')

print("결과 저장 완료:")
print(f"  - {output_dir}/시간대별_평균지표.csv")
print(f"  - {output_dir}/클러스터별_평균지표.csv")
print(f"  - {output_dir}/상권별_평균지표.csv")
print(f"  - {output_dir}/상권별_클러스터분포.csv")

결과 저장 완료:
  - /Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/5_보고서통계/시간대별_평균지표.csv
  - /Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/5_보고서통계/클러스터별_평균지표.csv
  - /Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/5_보고서통계/상권별_평균지표.csv
  - /Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/5_보고서통계/상권별_클러스터분포.csv


## 종합 요약

### 계산된 주요 지표

1. **시간대별 평균**:
   - 정상영업, D-12m, D-9m, D-6m, D-3m별 소속 상권 매출, 배달비율, 지하철승하차
   
2. **클러스터별 평균**:
   - 클러스터 0, 1, 2별 주요 지표
   - 고객 연령대 비중
   
3. **상권별 평균**:
   - 11개 상권별 접근성, 배달비율, 고객 특성
   - 고위험 비율

### 데이터 해석 주의사항

#### ✅ 정확한 해석:
- **'소속_상권_매출_억원'**: 가맹점이 속한 상권(상권_코드)의 월별 매출액
  - 같은 상권에 속한 모든 가맹점은 동일한 값을 가짐
  - 개별 가맹점의 실제 매출이 아님

#### ❌ 잘못된 해석:
- "가맹점의 평균 매출이 8억원이다" ← 틀림
- "개별 가맹점이 월 8억원을 벌어들인다" ← 틀림

#### ✅ 올바른 해석:
- "정상 가맹점들이 속한 상권의 월별 평균 매출은 8.14억원이다"
- "D-3m 가맹점들이 속한 상권의 월별 평균 매출은 10.08억원이다"

### 활용
- 보고서 작성 시 이 노트북의 출력 결과를 정확히 인용
- 모든 수치는 실제 계산 결과 기반
- 데이터 해석 시 항상 "소속 상권"임을 명시

---